In [ ]:
import gradio as gr
import uuid     # 임의 문자열 생성

In [ ]:
from chatbot_function import get_menu,get_order_price,set_order,compelet_order
from chatbot_responseV2  import get_first_chatbot_response,get_followup_chatbot_response,chat_with_bot

In [ ]:
from chatbot_function import menu,size_price

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🍕 피자 주문 챗봇")
    
    init_msg = "안녕하세요. 주문을 도와드리겠습니다.\n🤩메뉴를 확인해 주세요."
    chat_output = gr.Chatbot(label="PizzaOrderBot",type="messages",
                             value=[{"role":"assistant","content":init_msg}])
    user_input = gr.Textbox(placeholder="예: 슈퍼 슈프림 주문, 주문 완료", label="메시지 입력")
    submit = gr.Button("전송")

    # State 컴포넌트 : 앱 실행 중에 값을 유지하기 위해서 사용. 보이지는 않는 컴포넌트
    # 대화 내용 기록
    chat_history = gr.State([])   # [] 는 초기값
    # 주문 내용 기록
    orders = gr.State({"order_id": None, "content": [], "payment": 0})

     # 전송 버튼
    submit.click(
        chat_with_bot,
        inputs=[user_input, chat_history, orders], # orders 를 chat_with_bot 전달
        outputs=[user_input, chat_output, orders], # chat_with_bot 함수에 변경된 orders 리턴
    )
   # 입력창에서 엔터로 실행 
    user_input.submit(
        chat_with_bot,
        inputs=[user_input, chat_history, orders],
        outputs=[user_input, chat_output, orders],
    )

    with gr.Accordion("🍕 메뉴 보기"):
        gr.Markdown("### 📋 메뉴와 사이즈를 클릭하면 자동으로 입력창에 추가됩니다.")
        
        # 버튼 테이블 형태 (버튼으로 구성)
        with gr.Row():
            for name, price in menu.items():
                gr.Button(
                    f"{name} 💲 {price:,}원",
                    elem_id=f"btn_{name}",
                    scale=1,
                    min_width=180
                ).click(
                    None,
                    inputs=[],
                    outputs=[user_input],
                    js=f"() => (document.querySelector('textarea').value + ' {name} ')"
                )
        with gr.Row():
            for name, price in size_price.items():
                gr.Button(
                    f"{name} 💲 {price:,}원",
                    elem_id=f"btn_{name}",
                    scale=1,
                    min_width=180
                ).click(
                    None,
                    inputs=[],
                    outputs=[user_input],
                    js=f"() => (document.querySelector('textarea').value + ' {name} ')"
                )

demo.launch(inline=False,debug=True)

In [ ]:
demo.close()

#### 테스트 step1
- gpt 에게 보내는 함수의 요청을 텍스트에서 chat_history 로 변경

In [ ]:
'''
user_input ='피자 주문. 메뉴 보여주세요'
response = get_first_chatbot_response(user_input)
print(response.model_dump_json(indent=2))'
'''

#### 테스트 step2
- 함수 실행할 함수 이름 확인

In [ ]:
'''
import json
fn_name = getattr(response.choices[0].message.function_call, "name", None)
args = json.loads(response.choices[0].message.function_call.arguments)
print(f'🔄log fn_name: {fn_name} , args {args}')
'''

#### 테스트 step3
- 함수 실행

In [ ]:
'''
func_response =  globals()[fn_name](**args)
'''

#### 테스트 step4
- 함수 실행 결과로 gpt 응답 요청

In [ ]:
'''
followup = get_followup_chatbot_response(fn_name,func_response)
# print(followup.model_dump_json(indent=2))
print(followup.choices[0].message.content)
'''

In [ ]:
'''
# user_input ='메뉴를 보여주세요'
# user_input ='스파이스 치킨 주문 합니다.'
user_input ='미디움 주문합니다.'
response = get_chatbot_response(question=user_input)
print(response)
'''